## Question 1

**Bitmap Generation:**
There are several ways to generate the 7 x 12 bitmap image for each digit from the data. Please explain your approach and reasoning behind it.

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{\n    "username": "hannahbialik",\n    "key": "KGAT_46da0fae64a6eec1004b4f0c05c4de95"\n}'}

In [2]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d karnikakapoor/digits
!unzip digits.zip -d digits_data

Streaming output truncated to the last 5000 lines.
  inflating: digits_data/digits_jpeg/digits_jpeg/5/img006-00081.jpeg  
  inflating: digits_data/digits_jpeg/digits_jpeg/5/img006-00082.jpeg  
  inflating: digits_data/digits_jpeg/digits_jpeg/5/img006-00083.jpeg  
  inflating: digits_data/digits_jpeg/digits_jpeg/5/img006-00084.jpeg  
  inflating: digits_data/digits_jpeg/digits_jpeg/5/img006-00085.jpeg  
  inflating: digits_data/digits_jpeg/digits_jpeg/5/img006-00086.jpeg  
  inflating: digits_data/digits_jpeg/digits_jpeg/5/img006-00087.jpeg  
  inflating: digits_data/digits_jpeg/digits_jpeg/5/img006-00088.jpeg  
  inflating: digits_data/digits_jpeg/digits_jpeg/5/img006-00089.jpeg  
  inflating: digits_data/digits_jpeg/digits_jpeg/5/img006-00090.jpeg  
  inflating: digits_data/digits_jpeg/digits_jpeg/5/img006-00091.jpeg  
  inflating: digits_data/digits_jpeg/digits_jpeg/5/img006-00092.jpeg  
  inflating: digits_data/digits_jpeg/digits_jpeg/5/img006-00093.jpeg  
  inflating: digits_data/d

In [9]:
import os
import cv2
import numpy as np

root = "/content/digits_data/digits updated/digits updated"

bitmap_data = {}  # digit -> list of 7x12 bitmaps

for digit in sorted(os.listdir(root)):
    folder = os.path.join(root, digit)
    if not os.path.isdir(folder):
        continue

    bitmap_data[digit] = []

    for filename in os.listdir(folder):
        path = os.path.join(folder, filename)

        # --- 1. Load image ---
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue

        # --- 2. Normalize values to [0,1] ---
        img = img / 255.0

        # --- 3. Resize to 7 × 12 (height=7, width=12) ---
        resized = cv2.resize(img, (12, 7), interpolation=cv2.INTER_AREA)

        # --- 4. Convert to bitmap (binary) ---
        # Use Otsu thresholding
        _, bitmap = cv2.threshold((resized * 255).astype(np.uint8),
                                  0, 1, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        bitmap_data[digit].append(bitmap)

centers = np.zeros((10, 84))

for d in range(10):
    digit_str = str(d)
    bitmaps = bitmap_data[digit_str]  # list of 7x12 arrays

    # flatten each bitmap to 84-dim vector
    flattened = [b.flatten() for b in bitmaps]

    # compute center = mean vector
    centers[d] = np.mean(flattened, axis=0)

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [14]:
class GaussianRBF(nn.Module):
    """
    Implements the Gaussian RBF module used in the original LeNet-5 paper.
    - Takes an 84-dimensional input (flattened 7x12 bitmap).
    - Computes RBF similarity to 10 fixed class centers.
    - Outputs a 10-dimensional vector of RBF activations.
    """

    def __init__(self, centers, sigma=3.0):
        """
        centers: numpy array or torch tensor of shape (10, 84)
                 Each row is the center for one digit class.
        sigma:   Gaussian bandwidth parameter.
        """
        super().__init__()

        # Convert centers to tensor of shape (10,84)
        centers = torch.tensor(centers, dtype=torch.float32)

        # Register them as *buffers* (not learnable parameters)
        self.register_buffer("centers", centers)

        # Store σ² and factor for the RBF exponent
        self.sigma = sigma
        self.sigma_sq = sigma ** 2

    def forward(self, x):
        """
        x: tensor shape (batch_size, 84)
        Returns: tensor shape (batch_size, 10)
        """

        # Expand x to shape (batch_size, 1, 84)
        # Expand centers to shape (1, 10, 84)
        # Then squared difference broadcasts to (batch_size, 10, 84)
        diff = x.unsqueeze(1) - self.centers.unsqueeze(0)

        # Compute squared Euclidean distance
        dist_sq = torch.sum(diff ** 2, dim=2)  # -> (batch_size, 10)

        # Gaussian RBF activation
        rbf_out = torch.exp(-dist_sq / (2 * self.sigma_sq))

        return rbf_out

**Structure:**

In [15]:
class LearnedAvgPool(nn.Module): #S2 and S4
    def __init__(self, num_channels):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(num_channels))
        self.bias   = nn.Parameter(torch.zeros(num_channels))

    def forward(self, x):
        # Average pool 2×2 stride 2
        pooled = F.avg_pool2d(x, kernel_size=2, stride=2)

        # Apply learned α and β to each map
        w = self.weight.view(1, -1, 1, 1)
        b = self.bias.view(1, -1, 1, 1)
        return torch.tanh(w * pooled + b)

C3_CONNECTIONS = [
    [0,1,2], [1,2,3], [2,3,4], [3,4,5], [4,5,0], [5,0,1],
    [0,1,2,3], [1,2,3,4], [2,3,4,5], [3,4,5,0], [4,5,0,1], [5,0,1,2],
    [0,1,3,4], [1,2,4,5], [0,2,3,5],
    [0,1,2,3,4,5]
]

class LeNet5(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()

        # C1:
        # Input: 32×32×1
        # Output: 6 feature maps of size 28×28
        self.C1 = nn.Conv2d(1, 6, kernel_size=5)

        # S2:
        # Input: 6×28×28
        # Output: 6×14×14
        self.S2 = LearnedAvgPool(6)

        # C3:
        # Input: 6×14×14
        # Output: 16×10×10
        self.C3 = nn.ModuleList([
            nn.Conv2d(len(C3_CONNECTIONS[i]), 1, kernel_size=5)
            for i in range(16)
        ])

        # S4:
        # Input: 16×10×10
        # Output: 16×5×5
        self.S4 = LearnedAvgPool(16)

        # C5:
        # Input: 16×5×5
        # Output: 120
        self.C5 = nn.Conv2d(16, 120, kernel_size=5)

        # F6:
        # Input: 120
        # Output: 84
        self.F6 = nn.Linear(120, 84)

       # Gaussian RBF module (10 outputs)
        self.RBF = GaussianRBF(centers, sigma=3.0)

        # Final classifier: (84 CNN features + 10 RBF features) → 10
        self.final = nn.Linear(84 + 10, num_classes)

    def forward(self, x):

        # C1
        x = torch.tanh(self.C1(x))     # shape: N × 6 × 28 × 28

        # S2
        x = self.S2(x)                 # shape: N × 6 × 14 × 14

        # C3 (partial connections)
        C3_maps = []
        for i, conv in enumerate(self.C3):
            selected = x[:, C3_CONNECTIONS[i], :, :]  # pick channels
            C3_maps.append(torch.tanh(conv(selected)))

        x = torch.cat(C3_maps, dim=1)  # shape: N × 16 × 10 × 10

        # S4
        x = self.S4(x)                 # shape: N × 16 × 5 × 5

        # C5
        x = torch.tanh(self.C5(x))     # shape: N × 120 × 1 × 1
        x = x.view(x.size(0), -1)      # N × 120

        # F6
        f6 = torch.tanh(self.F6(x))     # N × 84

        # Output
        # RBF
        rbf_out = self.RBF(torch.xlogy_)       # N × 10

        # Concatenate CNN + RBF features
        combined = torch.cat([f6, rbf_out], dim=1)  # N × 94

        # Final classifier
        out = self.final(combined)   # N × 10

        return out


In [16]:
import pandas as pd
from PIL import Image

splits = {'train': 'mnist/train-00000-of-00001.parquet', 'test': 'mnist/test-00000-of-00001.parquet'}
df_train = pd.read_parquet("hf://datasets/ylecun/mnist/" + splits["train"])
df_test = pd.read_parquet("hf://datasets/ylecun/mnist/" + splits["test"])




/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
